<a href="https://colab.research.google.com/github/slippy0/gpt-2/blob/finetuning/gpt2_training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!git clone https://github.com/slippy0/gpt-2.git

fatal: destination path 'gpt-2' already exists and is not an empty directory.


In [0]:
%cd /content/gpt-2
!git checkout finetuning 
!pip install -r requirements.txt
%cd /content

/content/gpt-2
Already on 'finetuning'
Your branch is up to date with 'origin/finetuning'.
/content


In [0]:
# Settings

# 117M or 345M
model_name = "117M"

In [0]:
!python gpt-2/download_model.py $model_name

Fetching checkpoint: 1.00kit [00:00, 708kit/s]                                                      
Fetching encoder.json: 1.04Mit [00:00, 56.0Mit/s]                                                   
Fetching hparams.json: 1.00kit [00:00, 643kit/s]                                                    
Fetching model.ckpt.data-00000-of-00001: 498Mit [00:09, 51.2Mit/s]                                  
Fetching model.ckpt.index: 6.00kit [00:00, 2.84Mit/s]                                               
Fetching model.ckpt.meta: 472kit [00:00, 45.1Mit/s]                                                 
Fetching vocab.bpe: 457kit [00:00, 49.5Mit/s]                                                       


In [0]:
# UPLOAD .txt FILES TO /content FIRST
![ ! -d "./data" ] && mkdir data
%mv -- *.txt data

# Encode data
!PYTHONPATH=gpt-2/src gpt-2/encode.py \
    --model_name $model_name \
    data encoded_data.npz

Reading files
100% 2/2 [00:03<00:00,  2.17s/it]
Writing encoded_data.npz


In [0]:
# This runs forever. Stop manually when you're getting good samples
!PYTHONPATH=gpt-2/src gpt-2/train.py \
    --dataset "data/*.txt" \
    --model_name $model_name \
    --save_every 50 \
    --sample_every 50
#     --restore_from 'latest' \
#     --only_train_transformer_layers
#     --learning_rate 0.002

2019-05-28 05:38:29.182828: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2200000000 Hz
2019-05-28 05:38:29.183110: I tensorflow/compiler/xla/service/service.cc:150] XLA service 0x1ad0680 executing computations on platform Host. Devices:
2019-05-28 05:38:29.183148: I tensorflow/compiler/xla/service/service.cc:158]   StreamExecutor device (0): <undefined>, <undefined>
2019-05-28 05:38:29.398016: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2019-05-28 05:38:29.398662: I tensorflow/compiler/xla/service/service.cc:150] XLA service 0x1acffa0 executing computations on platform CUDA. Devices:
2019-05-28 05:38:29.398698: I tensorflow/compiler/xla/service/service.cc:158]   StreamExecutor device (0): Tesla T4, Compute Capability 7.5
2019-05-28 05:38:29.399098: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1433] Found de

In [0]:
# generate_samples is hardcoded to this path, so copy our trained models there
import glob, os
models = glob.glob('checkpoint/run1/model-*.meta')
latest = max(models, key=os.path.getctime)

base_name = os.path.splitext(os.path.basename(latest))[0]
print("Moving: " + base_name)
!cp checkpoint/run1/model${base_name}.index models/$model_name 
!cp checkpoint/run1/model${base_name}.meta models/$model_name 
!cp checkpoint/run1/model${base_name}.data-00000-of-00001 models/$model_name 
!cp checkpoint/run1/checkpoint models/$model_name 
!cp checkpoint/run1/counter models/$model_name 
# print(models)
# !cp -r /content/checkpoint/run1/* /content/models/$model_name/

Moving: model-1550


In [0]:
# Generate a random sample
!PYTHONPATH=gpt-2/src python gpt-2/src/generate_unconditional_samples.py \
    --model_name $model_name \
    --nsamples 5 \
    --top_p 0.95

2019-05-28 05:49:50.348107: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2200000000 Hz
2019-05-28 05:49:50.348389: I tensorflow/compiler/xla/service/service.cc:150] XLA service 0x2156520 executing computations on platform Host. Devices:
2019-05-28 05:49:50.348428: I tensorflow/compiler/xla/service/service.cc:158]   StreamExecutor device (0): <undefined>, <undefined>
2019-05-28 05:49:50.542805: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2019-05-28 05:49:50.543378: I tensorflow/compiler/xla/service/service.cc:150] XLA service 0x2155e40 executing computations on platform CUDA. Devices:
2019-05-28 05:49:50.543410: I tensorflow/compiler/xla/service/service.cc:158]   StreamExecutor device (0): Tesla T4, Compute Capability 7.5
2019-05-28 05:49:50.543763: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1433] Found de

In [0]:
# Generate samples with a prefix
!PYTHONPATH=gpt-2/src python gpt-2/src/interactive_conditional_samples.py \
    --model_name $model_name \
    --top_p 0.99

2019-05-25 23:33:40.857626: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2300000000 Hz
2019-05-25 23:33:40.857900: I tensorflow/compiler/xla/service/service.cc:150] XLA service 0x3198520 executing computations on platform Host. Devices:
2019-05-25 23:33:40.857934: I tensorflow/compiler/xla/service/service.cc:158]   StreamExecutor device (0): <undefined>, <undefined>
2019-05-25 23:33:41.045677: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2019-05-25 23:33:41.046225: I tensorflow/compiler/xla/service/service.cc:150] XLA service 0x3197e40 executing computations on platform CUDA. Devices:
2019-05-25 23:33:41.046260: I tensorflow/compiler/xla/service/service.cc:158]   StreamExecutor device (0): Tesla T4, Compute Capability 7.5
2019-05-25 23:33:41.046667: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1433] Found de

In [0]:
# Remove everything to start over
%rm -rf checkpoint
# %rm -rf data
%rm -rf models
%rm -rf gpt-2

In [0]:
%rm -rf *.txt